# $A^3$ Intermediate Problem Set 2 Solutions

This problem set will focus around binary search, which is *extremely* common in *USACO*. You can read about the different templates for binary search [here](https://leetcode.com/explore/learn/card/binary-search/).

## Question 1: [Leetcode 34. First, Last in Sorted Array](https://leetcode.com/problems/find-first-and-last-position-of-element-in-sorted-array/)
#### Lance B.
Direct application of templates, but keep in mind special cases.
```
class Solution {
public:
    int lowerBound(vector<int> &v, int target){
        if(v.size() == 0) return -1;

        int left = 0, right = v.size() - 1;
        while(left < right){
            int mid = left + (right - left) / 2;
            if(v[mid] < target){
                left = mid + 1;
            }
            else{
                right = mid;
            }
        }
        if(v[left] == target) return left;
        return -1;
    }

    int upperBound(vector<int> &v, int target){
        if(v.size() == 0) return -1;

        int left = 0, right = v.size() - 1;
        while(left < right){
            int mid = left + (right - left) / 2;
            if(v[mid] > target){
                right = mid;
            }
            else{
                left = mid + 1;
            }
        }
        if(v[left] == target) return left;
        else if (left > 0 && v[left - 1] == target) return left - 1;
        return -1;
    }
    vector<int> searchRange(vector<int>& nums, int target) {
        vector<int> ans;
        ans.push_back(lowerBound(nums, target));
        ans.push_back(upperBound(nums, target));
        return ans;
    }
};
```

## Question 2: [USACO 2018 Silver: Convention](http://www.usaco.org/index.php?page=viewproblem2&cpid=858)
For this question, do binary search to find the minimum possible wait time for the number of buses.
```
#include <iostream>
#include <fstream>
#include <vector>
#include <algorithm>
using namespace std;

int N, M, C;
vector<int> cows;

bool works(int wait){ // Simulate if a certain value works
    int idx = 0, busesUsed = 0, curr = 0;
    while(idx < cows.size()){
        while(curr - idx < C &&  curr < cows.size()){ // Fit as many cows as possible
            if(cows[curr] - cows[idx] > wait) break;
            curr++;
        }
        busesUsed++;
        idx = curr;
        if(busesUsed > M) return false;
    }
    return true;
}

int bs(){ // Binary search to find smallest value that works
    int left = 0, right = 1000000000;
    while(left < right){
        int mid = left + (right - left) / 2;
        if(!works(mid)) left = mid + 1;
        else right = mid;
    }
    return left;
}

int main(){
    ifstream fin ("convention.in");
    fin >> N >> M >> C;
    cows.resize(N);
    for(int i = 0; i < N; i++){
        fin >> cows[i];
    }
    sort(cows.begin(), cows.end());
    ofstream fout ("convention.out");
    fout << bs() << '\n';
}
```

## Question 3: [USACO 2020 Silver: Wormhole Sort](http://www.usaco.org/index.php?page=viewproblem2&cpid=992)
Binary search on the smallest width possible, and simulate DSU connectivity. I'll leave the following block for you to dissect.
```
#include <iostream>
#include <fstream>
#include <vector>
#include <algorithm>
using namespace std;

typedef vector<int> vi;
typedef vector<vi> vvi;
typedef long long ll;
typedef vector<ll> vll;
typedef pair<int, int> pii;
typedef pair<int, pii> pipii;
typedef vector<pipii> vpipii;
typedef vector<char> vc;
typedef vector<vc> vvc;

#define PB push_back

class dsu{
  public:
  vll arr, size;
  int cc;
  vi cost;

  dsu(int N){
    arr.resize(N, 0);
    for(int i = 0; i < N; i++) arr[i] = i;
    size.resize(N, 1);
    cc = N;
  }

  bool find(int A, int B){
    return root(A) == root(B);
  }

  void connect(int A, int B){
    int rootA = root(A), rootB = root(B);
    if(rootA != rootB){
      cc--;
      if(size[rootA] < size[rootB]){
        arr[rootA] = rootB;
        size[rootB] += size[rootA];
      }
      else{
        arr[rootB] = rootA;
        size[rootA] += size[rootB];
      }
    }
  }

  void add(int A){
    size[A] = 1;
    cc++;
  }

  void print_roots(){
    for(int i : arr) cout << i << ' ';
    cout << '\n';
  }

  int getCC(){
    return cc;
  }

  bool exists(int A){
    return size[A] > 0;
  }

  private:
  int root(int A){
    if(A == arr[A]){
      return A;
    }
    arr[A] = root(arr[A]);
    return arr[A];
  }
};

bool works(int biggest, vpipii &wormholes, vi &order){
  int i = 0;
  dsu myDSU (order.size());
  while(true){
    if(i == wormholes.size() || wormholes[i].first < biggest) break;
    myDSU.connect(wormholes[i].second.first, wormholes[i].second.second);
    i++;
  }
  for(int i = 0; i < order.size(); i++){
    if(!myDSU.find(i, order[i])){
      return false;
    }
  }
  return true;
}

int main(){
  ifstream fin ("wormsort.in");
  ofstream fout ("wormsort.out");
  int N, M; fin >> N >> M;
  vi order; order.resize(N);
  bool sorted = true;
  for(int i = 0; i < N; i++){
    fin >> order[i];
    order[i]--;
    if(i != order[i]) sorted = false;
  }
  if(sorted) fout << -1 << '\n';
  else{
    vpipii wormholes (M);
    for(int i = 0; i < M; i++){
      fin >> wormholes[i].second.first >> wormholes[i].second.second >> wormholes[i].first;
      wormholes[i].second.second--; wormholes[i].second.first--;
    }
    sort(wormholes.begin(), wormholes.end(), greater<pipii>());
    int left = 0, right = M - 1;
    while(left < right){
      int mid = left + (right - left) / 2;
      if(!works(wormholes[mid].first, wormholes, order)){
        left = mid + 1;
      }
      else{
        right = mid;
      }
    }
    fout << wormholes[left].first << '\n';
  }
}
```

#### Formatting

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))